In [1]:
from mlflow.tracking import MlflowClient

MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [3]:
client.create_experiment(name="my-cool-experiment")

'2'

In [11]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='1',
    filter_string="metrics.rmse < 6.38",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"]

)

In [12]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']}:.4f")

run id: 2b597f1c25b44ad293f9981dd9ffa1b0, rmse: 6.3071136777830965:.4f
run id: c19b3543904a4de493dbf9b65efa03d3, rmse: 6.337520397166299:.4f
run id: 4a155821e80a49d7a830c240730572a6, rmse: 6.361311942705955:.4f


In [14]:
import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [21]:
run_id = "2b597f1c25b44ad293f9981dd9ffa1b0"
model_uri = f"runs:/{run_id}/model"
model_name = "nyc-taxi-model"

mlflow.register_model(model_uri=model_uri, 
                      name=model_name)

Registered model 'nyc-taxi-model' already exists. Creating a new version of this model...
Created version '4' of model 'nyc-taxi-model'.


<ModelVersion: aliases=[], creation_timestamp=1747906003505, current_stage='None', description=None, last_updated_timestamp=1747906003505, name='nyc-taxi-model', run_id='2b597f1c25b44ad293f9981dd9ffa1b0', run_link=None, source=('/Users/steffi/Documents/MLOps '
 'Zoomcamp/mlops-zoomcamp/02-experiment-tracking/lesson/src/mlruns/1/2b597f1c25b44ad293f9981dd9ffa1b0/artifacts/model'), status='READY', status_message=None, tags={}, user_id=None, version=4>

In [25]:
versions = client.get_latest_versions(name=model_name)
for version in versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 4, stage: None


/var/folders/pm/qyg6bv9j25q7zp4_b3mt8j_c0000gn/T/ipykernel_36775/760175356.py:1: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  versions = client.get_latest_versions(name=model_name)


In [26]:
client.transition_model_version_stage(
    name=model_name,
    version=1,
    stage="Staging",
    archive_existing_versions=False
)

/var/folders/pm/qyg6bv9j25q7zp4_b3mt8j_c0000gn/T/ipykernel_36775/2608318267.py:1: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1747904514090, current_stage='Staging', description='', last_updated_timestamp=1747906174535, name='nyc-taxi-model', run_id='7394a5dc0fbc4216bb14b4880f0eff82', run_link='', source=('/Users/steffi/Documents/MLOps '
 'Zoomcamp/mlops-zoomcamp/02-experiment-tracking/lesson/src/mlruns/1/7394a5dc0fbc4216bb14b4880f0eff82/artifacts/models_mlflow'), status='READY', status_message=None, tags={'model': 'xgboost'}, user_id=None, version=1>

In [30]:
from datetime import datetime

date = datetime.today().date

client.update_model_version(
    name=model_name,
    version=1,
    description=f"Transitioned to staging on {date}"
)

<ModelVersion: aliases=[], creation_timestamp=1747904514090, current_stage='Staging', description=('Transitioned to staging on <built-in method date of datetime.datetime object '
 'at 0x147715ed0>'), last_updated_timestamp=1747906585816, name='nyc-taxi-model', run_id='7394a5dc0fbc4216bb14b4880f0eff82', run_link='', source=('/Users/steffi/Documents/MLOps '
 'Zoomcamp/mlops-zoomcamp/02-experiment-tracking/lesson/src/mlruns/1/7394a5dc0fbc4216bb14b4880f0eff82/artifacts/models_mlflow'), status='READY', status_message=None, tags={'model': 'xgboost'}, user_id=None, version=1>

In [45]:
from sklearn.metrics import root_mean_squared_error
import pandas as pd


def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df


def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)


def test_model(name, stage, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}/{stage}")
    y_pred = model.predict(X_test)
    return {"rmse": root_mean_squared_error(y_test, y_pred)}

In [35]:
df = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-03.parquet')

In [38]:
client.download_artifacts(run_id="aa59e96d8cd1459f891304638e754672", path='preprocessor', dst_path=".")

'/Users/steffi/Documents/MLOps Zoomcamp/mlops-zoomcamp/02-experiment-tracking/lesson/src/preprocessor'

In [40]:
import pickle

with open("preprocessor/preprocessor.b", "rb") as f_in:
    dv = pickle.load(f_in)

In [41]:
x_test = preprocess(df, dv)
target = "duration"
y_test = df[target].values

In [46]:
%time test_model(name=model_name, stage="Production", X_test=x_test, y_test=y_test)

CPU times: user 102 ms, sys: 27.9 ms, total: 130 ms
Wall time: 155 ms


{'rmse': 6.659623830022514}

In [48]:
%time test_model(name=model_name, stage="Staging", X_test=x_test, y_test=y_test)

CPU times: user 205 ms, sys: 39.6 ms, total: 245 ms
Wall time: 367 ms


{'rmse': 10.810679150003407}

In [49]:
client.transition_model_version_stage(
    name=model_name,
    version=1,
    stage="Production",
    archive_existing_versions=True,
)

/var/folders/pm/qyg6bv9j25q7zp4_b3mt8j_c0000gn/T/ipykernel_36775/4018881539.py:1: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1747904514090, current_stage='Production', description=('Transitioned to staging on <built-in method date of datetime.datetime object '
 'at 0x147715ed0>'), last_updated_timestamp=1747907611469, name='nyc-taxi-model', run_id='7394a5dc0fbc4216bb14b4880f0eff82', run_link='', source=('/Users/steffi/Documents/MLOps '
 'Zoomcamp/mlops-zoomcamp/02-experiment-tracking/lesson/src/mlruns/1/7394a5dc0fbc4216bb14b4880f0eff82/artifacts/models_mlflow'), status='READY', status_message=None, tags={'model': 'xgboost'}, user_id=None, version=1>